In [42]:
import pymysql.cursors
import pandas as pd
from datetime import datetime, timedelta
import calendar
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [2]:
#import pyodbc
#conn = pyodbc.connect("DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={0}; database={1};UID={2};PWD={3}".format('10.20.2.110,1433','AIG_DARAZ_PK_DW','daraz-tools','JfF7ziSCNk5jPxd'))
#cursor = conn.cursor()

#query = open('SqlForDailyDemandData.sql', 'r')
#dataFrame_initial = pd.read_sql_query(query.read(), conn)
#dataFrame_initial.shape

In [3]:
#dataFrame_initial.to_pickle('./dataFile',compression='infer', protocol=4)

In [4]:
df_initial = pd.read_pickle('./dataFile', compression='infer')

In [5]:
#df_initial.dtypes

In [6]:
#query = open('ProductVisibility.sql', 'r')
#df_prodVisible = pd.read_sql_query(query.read(), conn)
#df_prodVisible.shape

In [5]:
#df_prodVisible.to_pickle('./productVisibilityDataFile',compression='infer', protocol=4)
df_prodVisible = pd.read_pickle('./productVisibilityDataFile', compression='infer')


In [6]:
df_initial['date'] = pd.to_datetime(df_initial['date'])
df_initial.sort_values(by='date').head()

,COD_SKU_CONFIG,date,DSC_CATEGORY_NAME_LEVEL_1,DSC_CATEGORY_NAME_LEVEL_2,DSC_CATEGORY_NAME_LEVEL_3,DSC_CATEGORY_NAME_LEVEL_4,DSC_PRODUCT_NAME,DSC_PRODUCT_BRAND_NAME_EN,DSC_PRODUCT_SIZE,DSC_PRODUCT_COLOR,DSC_PRODUCT_WEIGHT,DSC_SHIPPING_METHOD,Android,Desktop,iOS,Mobile (Theme),Amount,Valid Total
1066686,RO711EL1LP57QNAFAMZ,2017-12-01,Phones & Tablets,Phone & Tablet Accessories,Power Banks,7100 - 10000+ mAh Power Bank,Solo 3 Powerbank With Light- 6000mAh - White,Romoss,,White,0.5,UniversalShippingMatrix,1,0,0,0,968.0,1
548709,HA779HL0DDK8GNAFAMZ,2017-12-01,Appliances,Large Appliances,Cooling,Cooling,HSU-18LK10/L011 - Split Ac - 1.5 Ton - White,Haier - Official,,White,60,UniversalShippingMatrix,1,0,0,0,39578.0,1
1276971,TH134EL0L0D7YNAFAMZ,2017-12-01,Phones & Tablets,Phone & Tablet Accessories,Cases & Covers,Iphone & Ipad,Iphone X Premium Jelly Case - Transparent,Daraz Accessories,,Neutral,0.2,UniversalShippingMatrix,0,0,1,0,149.0,1
398560,EA835EL0WDZIMNAFAMZ,2017-12-01,Phones & Tablets,Phone & Tablet Accessories,Other Accessories,Others,Universal Mobile Phone GPS Car Holder 360 Degr...,Earldom,,Yellow & Black,0.2,UniversalShippingMatrix,1,0,0,0,750.0,1
271913,CM114EL0TWC9SNAFAMZ,2017-12-01,Phones & Tablets,Land Line,Corded,N/A,LCD Touch Panel Telephone With 8 Digit Calcula...,CM,,N/A,0.5,UniversalShippingMatrix,1,1,0,0,1099.0,2


In [9]:
#df_initial['DSC_PRODUCT_WEIGHT'].unique()

In [10]:
import re
df_initial['DSC_PRODUCT_WEIGHT'] = df_initial['DSC_PRODUCT_WEIGHT'].apply(lambda x : re.sub(r'#', r'0', str(x)))
df_initial['DSC_PRODUCT_WEIGHT'] = df_initial['DSC_PRODUCT_WEIGHT'].apply(lambda x : re.sub(r'[a-zA-Z]', '', str(x)))
df_initial['DSC_PRODUCT_WEIGHT'] = df_initial['DSC_PRODUCT_WEIGHT'].apply(lambda x : re.sub(r'([.])\1+', r'\1', str(x)))
df_initial['DSC_PRODUCT_WEIGHT'] = df_initial['DSC_PRODUCT_WEIGHT'].apply(lambda x : re.sub(r'/', r'', str(x)))
df_initial['DSC_PRODUCT_WEIGHT'] = df_initial['DSC_PRODUCT_WEIGHT'].apply(lambda x : re.sub(r'[!]', '0', str(x)))
df_initial['DSC_PRODUCT_WEIGHT'] = pd.to_numeric(df_initial['DSC_PRODUCT_WEIGHT'])


In [11]:
#df_initial.describe()

In [12]:
#df_initial[df_initial.DSC_PRODUCT_WEIGHT == df_initial['DSC_PRODUCT_WEIGHT'].max()]

In [8]:
df_initial['CatConcat'] = df_initial[['DSC_CATEGORY_NAME_LEVEL_1', 'DSC_CATEGORY_NAME_LEVEL_2','DSC_CATEGORY_NAME_LEVEL_3','DSC_CATEGORY_NAME_LEVEL_4']].apply(lambda x: ' '.join(x.str.strip()), axis=1)

In [9]:
#df_initial[df_initial.CatConcat == "Grocer's Shop Cooking Essentials Rice, Flours, Pulses & Grains Rice"]['DSC_PRODUCT_NAME'].value_counts()[:10]

In [10]:
# df_initial[df_initial.DSC_PRODUCT_NAME == 'Super Kernel Basmati Rice']['COD_SKU_CONFIG'].value_counts()

In [11]:
# df_initial[(df_initial.DSC_PRODUCT_NAME == 'Super Kernel Basmati Rice') &
#            (df_initial.DSC_PRODUCT_BRAND_NAME_EN == 'Xidmat')]['date'].max()

In [12]:
# newDf = df_initial[(df_initial.COD_SKU_CONFIG == 'RE775OT1GWF5GNAFAMZ')]['date'].to_frame()

# newDf['CummulativeQuantity'] = df_initial[(df_initial.COD_SKU_CONFIG == 'RE775OT1GWF5GNAFAMZ')]['Valid Total'].cumsum()

# newDf['DemandPercent'] = (newDf['CummulativeQuantity']/newDf['CummulativeQuantity'].iloc[-1])*100

In [13]:
#df_initial[(df_initial.COD_SKU_CONFIG == 'RE775OT1GWF5GNAFAMZ')].head()

In [14]:
#newDf[40:105]

In [15]:
# plt.figure(figsize=(10,7))
# plt.plot(newDf.date, newDf.DemandPercent)
# plt.ylabel('Percentage Demand', fontsize=12)
# plt.xlabel('Date', fontsize=12)

In [16]:
df_prodVisible = df_prodVisible.fillna(0)

df_prodVisible = df_prodVisible.T

In [17]:
df_prodVisible.columns = df_prodVisible.iloc[0]

df_prodVisible = df_prodVisible.reindex(df_prodVisible.index.drop('DD_COD_SKU_CONFIG'))

df_prodVisible.head(5)

DD_COD_SKU_CONFIG,00301FA0IW7BMNAFAMZ,00301FA0QSN4YNAFAMZ,00301FA0WQZ6QNAFAMZ,00301FA1IM7MANAFAMZ,10259FA000PQYNAFAMZ,10259FA002UWQNAFAMZ,10259FA0074AYNAFAMZ,10259FA00FGGINAFAMZ,10259FA00LZEANAFAMZ,10259FA00M5CQNAFAMZ,...,ZZ970FA12FBTMNAFAMZ,ZZ970FA138WHENAFAMZ,ZZ970FA153S1MNAFAMZ,ZZ970FA15MMWINAFAMZ,ZZ970FA17S89MNAFAMZ,ZZ970FA1BKYYANAFAMZ,ZZ970FA1E9URENAFAMZ,ZZ970FA1GESO2NAFAMZ,ZZ970FA1KKTWINAFAMZ,ZZ970FA1NHN1UNAFAMZ
2017-12-01,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2017-12-02,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2017-12-03,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2017-12-04,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2017-12-05,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
def zero_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges

In [19]:
def getOfflineData(offlineRaw,SKU_name):
    dataFrame = pd.DataFrame()
    for ind, column in enumerate(offlineRaw.columns):
        runs = zero_runs(offlineRaw.iloc[:,ind])
        tuples = zip(offlineRaw.iloc[runs[:,0],0].index, runs[:,1] - runs[:,0])
        tempDf = {}
        cols = ['OfflineDate','OfflineDays']
        tuples = list(tuples)
        tempDf = pd.DataFrame.from_records(tuples,columns = cols)
        tempDf['SKU'] = column
        dataFrame = dataFrame.append(tempDf)
    return dataFrame
        

In [20]:
#formattedDf = getOfflineData(df_prodVisible,df_prodVisible.columns.values[0])
#formattedDf.to_pickle('./OfflineFormattedData',compression='infer', protocol=4)
formattedDf = pd.read_pickle('./OfflineFormattedData', compression='infer')
#formattedDf.head()

In [21]:
#Sample SKUs 00301FA0IW7BMNAFAMZ, 00301FA1IM7MANAFAMZ, 10259FA0074AYNAFAMZ
#formattedDf['OfflineDate'] = pd.to_datetime(formattedDf['OfflineDate'])
#df_initial.dtypes

### Offline Product Similarity Match 

In [22]:
SKUid = '10259FA1DDLKCNAFAMZ'
PeriodStartDate = '2018-03-04'
categoryName = df_initial[df_initial.COD_SKU_CONFIG == SKUid]['CatConcat'].unique()
print(categoryName)
productame = df_initial[df_initial.COD_SKU_CONFIG == SKUid]['DSC_PRODUCT_NAME'].unique()
print(productame)
brandName = df_initial[df_initial.COD_SKU_CONFIG == SKUid]['DSC_PRODUCT_BRAND_NAME_EN'].unique()
print(brandName)
offlineStartDate = formattedDf[formattedDf.SKU == SKUid].iloc[3:4,0].values[0]
print(offlineStartDate)

offlineDays = formattedDf[formattedDf.SKU == SKUid].iloc[3:4,1].values[0]
print(offlineDays)

relevantSkuList = df_initial[(df_initial.CatConcat == categoryName[0]) & (df_initial.COD_SKU_CONFIG != SKUid)]['COD_SKU_CONFIG'].unique()
print('SKUs which have been sold during this period from this relevent category cluster ',len(relevantSkuList))

#print('No of records in this Category, whose demand data we have in our date range of 5 months:',len(df_initial[(df_initial.CatConcat == categoryName[0]) & (df_initial.COD_SKU_CONFIG != '00301FA0IW7BMNAFAMZ')].index))


["Women's Fashion Women's Accessories Jewellery Body Piercings"]
['Metal Body Piercing Set']
['100 Degreez']
2018-03-06
1
SKUs which have been sold during this period from this relevent category cluster  35


In [23]:
#df_prodVisible[np.intersect1d(df_prodVisible.columns, list(relevantSkuList))].head()
#df_prodVisible[df_prodVisible.columns.intersection(list(relevantSkuList))].head()

In [24]:
releventSKUdataFrame = df_prodVisible[(datetime.strptime(PeriodStartDate, "%Y-%m-%d").strftime("%Y-%m-%d")) : (datetime.strptime(offlineStartDate, "%Y-%m-%d") + timedelta(days=int(offlineDays)-1)).strftime("%Y-%m-%d")]

releventSKUdataFrame = releventSKUdataFrame[releventSKUdataFrame.columns.intersection(list(relevantSkuList))].apply(pd.Series.value_counts)

In [25]:
releventSKUdataFrame

DD_COD_SKU_CONFIG,10259FA03W2Y4NAFAMZ,10259FA04HIJWNAFAMZ,10259FA05WYEKNAFAMZ,10259FA0JDCOCNAFAMZ,10259FA0RRKB0NAFAMZ,10259FA0VA0RWNAFAMZ,10259FA0XNR70NAFAMZ,10259FA0XPWCSNAFAMZ,10259FA1152M4NAFAMZ,10259FA15C8WCNAFAMZ,...,HE632FA121Y6QNAFAMZ,JM049FA1HLVQYNAFAMZ,LA958FA0B6HPGNAFAMZ,LA958FA0ZBQZSNAFAMZ,MA501FA05Q4YQNAFAMZ,MA501FA13XL20NAFAMZ,RH255FA0R1304NAFAMZ,TO118FA1C29XSNAFAMZ,ZA401FA0CZFLCNAFAMZ,ZA401FA0N9DSWNAFAMZ
0.0,1,1,1,1,1,1,1,1,1,1,...,1,3.0,1,1,1,1,3.0,3.0,3.0,1
1.0,2,2,2,2,2,2,2,2,2,2,...,2,NaN,2,2,2,2,NaN,NaN,NaN,2


In [26]:
findOnlineRelatedSKUs = releventSKUdataFrame

In [27]:
findOnlineRelatedSKUs = findOnlineRelatedSKUs.T

In [28]:
if findOnlineRelatedSKUs.shape[1] == 2:
     findOnlineRelatedSKUs.columns = ['Off', 'On']
else:
     findOnlineRelatedSKUs.columns = ['On']

In [29]:
findOnlineRelatedSKUs.shape

(35, 2)

In [30]:
#Get SKUs from the smilar valid SKU list which were OnLine atleast
similarSKU4DC = findOnlineRelatedSKUs[findOnlineRelatedSKUs["On"] > 0].index
len(similarSKU4DC)

24

In [31]:
demandCurveDf = df_initial[df_initial.COD_SKU_CONFIG.isin(similarSKU4DC)]
demandCurveDf = demandCurveDf[(demandCurveDf.date >= datetime.strptime(PeriodStartDate, "%Y-%m-%d")) 
             & (demandCurveDf.date <= datetime.strptime(offlineStartDate, "%Y-%m-%d") + timedelta(days=int(offlineDays)-1))]
#demandCurveDf = demandCurveDf['date'].to_frame()
demandCurveDf.shape

(0, 19)

In [32]:
if demandCurveDf.shape[0] > 0:
    temp = pd.DataFrame(demandCurveDf.groupby(['date'])['Valid Total'].mean()).reset_index()
    demandCurveDf = temp['date'].to_frame()
    demandCurveDf['CummulativeQuantity'] = temp['Valid Total'].cumsum().round(1)
    demandCurveDf['DemandPercent'] = (demandCurveDf['CummulativeQuantity']/demandCurveDf['CummulativeQuantity'].iloc[-1])*100

In [33]:
demandCurveDf.tail(50)

,COD_SKU_CONFIG,date,DSC_CATEGORY_NAME_LEVEL_1,DSC_CATEGORY_NAME_LEVEL_2,DSC_CATEGORY_NAME_LEVEL_3,DSC_CATEGORY_NAME_LEVEL_4,DSC_PRODUCT_NAME,DSC_PRODUCT_BRAND_NAME_EN,DSC_PRODUCT_SIZE,DSC_PRODUCT_COLOR,DSC_PRODUCT_WEIGHT,DSC_SHIPPING_METHOD,Android,Desktop,iOS,Mobile (Theme),Amount,Valid Total,CatConcat


In [34]:
temp = df_initial[df_initial.COD_SKU_CONFIG == SKUid]
temp = temp[(temp.date >= datetime.strptime(PeriodStartDate, "%Y-%m-%d")) 
                     & (temp.date <= datetime.strptime(offlineStartDate, "%Y-%m-%d") + timedelta(days=int(offlineDays)-1))]

if temp['COD_SKU_CONFIG'].count() > 0:
    onlineDemand = 1
    demandCurveDf4Offline = temp['date'].to_frame()
    demandCurveDf4Offline['CummulativeQuantity'] = temp[(temp.COD_SKU_CONFIG == SKUid)]['Valid Total'].cumsum()
    demandCurveDf4Offline['DemandPercent'] = (demandCurveDf4Offline['CummulativeQuantity']/demandCurveDf4Offline['CummulativeQuantity'].iloc[-1])*100
    print(demandCurveDf4Offline)
    print("Recorded demand during Online period : ",temp['COD_SKU_CONFIG'].count())
else:
    onlineDemand = 0
    print("No Demand Recorded during Online period")

           date  CummulativeQuantity  DemandPercent
1296 2018-03-05                    1          100.0
Recorded demand during Online period :  1


In [ ]:
if onlineDemand > 0:
    dateOfSoldOut = demandCurveDf4Offline[demandCurveDf4Offline.DemandPercent == 100]['date'].values[0]
    DemandPercentWhenSoldOut = demandCurveDf[demandCurveDf.date >= dateOfSoldOut][0:1]['DemandPercent']/100
    if DemandPercentWhenSoldOut.count() == 0:
        print("Estiamted Lost Sales (Category SO) :",DemandPercentWhenSoldOut.count())
    else:
        CummulativeQuantityWhenSoldOut = demandCurveDf4Offline[demandCurveDf4Offline.DemandPercent == 100]['CummulativeQuantity']
        print("Cummulative Quantity When Sold Out:",CummulativeQuantityWhenSoldOut.values[0])
        print("Demand Percent of Category when the product soldout ",DemandPercentWhenSoldOut.values[0])
        EstiamtedLostSales = CummulativeQuantityWhenSoldOut/DemandPercentWhenSoldOut.values[0]
        print("Estiamted Lost Sales :",(EstiamtedLostSales.values[0] - CummulativeQuantityWhenSoldOut).values[0])

In [ ]:
plt.figure(figsize=(10,7))
markers_on = [DemandPercentWhenSoldOut.index[0]]
plt.plot(demandCurveDf.date, demandCurveDf.DemandPercent,'-bX',markevery=markers_on)

for a,b in zip(demandCurveDf.date, demandCurveDf.DemandPercent): 
    if( b == DemandPercentWhenSoldOut.values[0] * 100):
        plt.text(a, b, "Category Demand % : " + str(b) + "  \nDate : " + str(a))
        
plt.ylabel('Percentage Demand', fontsize=12)
plt.xlabel('Date', fontsize=12)

In [37]:
def getEstimatedLostSale(SKUid,startDate,offlineStartDate,offlineDays):
    
    #print("---------------------------------------------------------------------------")
    #print("SKU : ",SKUid ," Start Date ",startDate, " OfflineDate ", offlineStartDate , " Offline for Days ", offlineDays)
             
    PeriodStartDate = startDate
    
    skuDemandCheckDf = df_initial[df_initial.COD_SKU_CONFIG == SKUid]

    skuDemandCheckDf = skuDemandCheckDf[(skuDemandCheckDf.date >= datetime.strptime(PeriodStartDate, "%Y-%m-%d")) 
                     & (skuDemandCheckDf.date <= datetime.strptime(offlineStartDate, "%Y-%m-%d") + timedelta(days=int(offlineDays)-1))]

    if PeriodStartDate == offlineStartDate:
        return "Product Offline : " + str(datetime.strptime(startDate, "%Y-%m-%d")) + " - " + str(datetime.strptime(offlineStartDate, "%Y-%m-%d") + timedelta(days=int(offlineDays)-1))
 
    elif skuDemandCheckDf['COD_SKU_CONFIG'].count() == 0:
        return "No Demand : " + str(datetime.strptime(startDate, "%Y-%m-%d")) + " - " + str(datetime.strptime(offlineStartDate, "%Y-%m-%d") + timedelta(days=int(offlineDays)-1))
    else:
        
        categoryName = df_initial[df_initial.COD_SKU_CONFIG == SKUid]['CatConcat'].unique()
        #print("Category Concatented Name : ",categoryName)
        productame = df_initial[df_initial.COD_SKU_CONFIG == SKUid]['DSC_PRODUCT_NAME'].unique()
       
        brandName = df_initial[df_initial.COD_SKU_CONFIG == SKUid]['DSC_PRODUCT_BRAND_NAME_EN'].unique()
        #print("Brand Name : ",brandName)
        
        relevantSkuList = df_initial[(df_initial.CatConcat == categoryName[0]) & (df_initial.COD_SKU_CONFIG != SKUid)]['COD_SKU_CONFIG'].unique()
        #print('Total Unique SKUs from this relevent category',len(relevantSkuList))
        
        releventSKUdataFrame = df_prodVisible[(datetime.strptime(PeriodStartDate, "%Y-%m-%d").strftime("%Y-%m-%d")) : (datetime.strptime(offlineStartDate, "%Y-%m-%d") + timedelta(days=int(offlineDays)-1)).strftime("%Y-%m-%d")]

        releventSKUdataFrame = releventSKUdataFrame[releventSKUdataFrame.columns.intersection(list(relevantSkuList))].apply(pd.Series.value_counts)
        
        findOnlineRelatedSKUs = releventSKUdataFrame
        
        findOnlineRelatedSKUs = findOnlineRelatedSKUs.T
        
        if findOnlineRelatedSKUs.shape[1] == 2:
             findOnlineRelatedSKUs.columns = ['Off', 'On']
        else:
             findOnlineRelatedSKUs.columns = ['On']
        #Get SKUs from the smilar valid SKU list which were OnLine atleast
        similarSKU4DC = findOnlineRelatedSKUs[findOnlineRelatedSKUs["On"] > 0].index
        #print("Potential SKUs in the category to match with (Online Ones) : ", len(similarSKU4DC))
        
        
        demandCurveDf = df_initial[df_initial.COD_SKU_CONFIG.isin(similarSKU4DC)]
        demandCurveDf = demandCurveDf[(demandCurveDf.date >= datetime.strptime(PeriodStartDate, "%Y-%m-%d")) 
             & (demandCurveDf.date <= datetime.strptime(offlineStartDate, "%Y-%m-%d") + timedelta(days=int(offlineDays)-1))]
        
        ##When No data found for the relevent Category of the SKU of search
        if demandCurveDf.shape[0] == 0:
            return '0'
        
        temp = pd.DataFrame(demandCurveDf.groupby(['date'])['Valid Total'].mean()).reset_index()
        demandCurveDf = temp['date'].to_frame()
        demandCurveDf['CummulativeQuantity'] = temp['Valid Total'].cumsum().round(1)
        demandCurveDf['DemandPercent'] = (demandCurveDf['CummulativeQuantity']/demandCurveDf['CummulativeQuantity'].iloc[-1])*100
        demandCurveDf
        
        #####Demand Curve for Offlined SKU
        demandCurveDf4Offline = skuDemandCheckDf['date'].to_frame()
        demandCurveDf4Offline['CummulativeQuantity'] = skuDemandCheckDf[(skuDemandCheckDf.COD_SKU_CONFIG == SKUid)]['Valid Total'].cumsum()
        demandCurveDf4Offline['DemandPercent'] = (demandCurveDf4Offline['CummulativeQuantity']/demandCurveDf4Offline['CummulativeQuantity'].iloc[-1])*100
        demandCurveDf4Offline
        
        dateOfSoldOut = demandCurveDf4Offline[demandCurveDf4Offline.DemandPercent == 100]['date'].values[0]
        demandCurveDf4Offline
        DemandPercentWhenSoldOut = demandCurveDf[demandCurveDf.date >= dateOfSoldOut][0:1]['DemandPercent']/100
        
        ## for case when the Product Category last sold was lesser then the product SKU itself , so we can assume that 
        ## there is no LOST sale , since SKU was sold after the category Overall was already been soldout .
        ## example SKU and dates are 10259FA0924GYNAFAMZ and 4:5 Offline Record 
        
        if DemandPercentWhenSoldOut.count() == 0:
            return '0';
        
        CummulativeQuantityWhenSoldOut = demandCurveDf4Offline[demandCurveDf4Offline.DemandPercent == 100]['CummulativeQuantity']
        
        EstiamtedLostSales = CummulativeQuantityWhenSoldOut/DemandPercentWhenSoldOut.values[0]
        
#         print(" Cummulative Quantity :",CummulativeQuantityWhenSoldOut.values[0], " Demand Percent of Category : ",DemandPercentWhenSoldOut.values[0],
#               " Estiamted Lost Sales :", (EstiamtedLostSales.values[0] - CummulativeQuantityWhenSoldOut).values[0],
#               " Product Name : ",productame)
       
        return (EstiamtedLostSales.values[0] - CummulativeQuantityWhenSoldOut).values[0]


In [36]:
getEstimatedLostSale(SKUid,PeriodStartDate,offlineStartDate,offlineDays)

---------------------------------------------------------------------------
SKU :  10259FA1DDLKCNAFAMZ  Start Date  2018-03-04  OfflineDate  2018-03-06  Offline for Days  1


'0'

In [47]:
startDate = None
previousSKU = None
currentSKU = None
estLostSale = []
endDate = None 
baseStartDate = "2017-12-01"
import time;

localtime = time.time()
print("Local current time :", localtime)
for i, row in formattedDf[:500].iterrows():
    currentSKU = row['SKU']
    if previousSKU == None:
        previousSKU = currentSKU
    elif previousSKU != currentSKU:
        startDate = None
        previousSKU = currentSKU
        #print("SKU Changed .. Updating SKU")
    
    if startDate == None:
        startDate = datetime.strptime(baseStartDate, "%Y-%m-%d").strftime("%Y-%m-%d")
        #print("Start : ",startDate)
        endDate = datetime.strptime(row['OfflineDate'], "%Y-%m-%d") + timedelta(days=int(row['OfflineDays'])-1)
        #print("End : ", endDate)
    else:
        startDate = endDate + timedelta(days=int(1))
        #print("Start : ",startDate)
        delta = startDate - (datetime.strptime(row['OfflineDate'], "%Y-%m-%d") + timedelta(days=int(row['OfflineDays'])-1))
        delta = delta.days * -1
        #print("Delta : ", delta)
        endDate = (datetime.strptime(startDate.strftime("%Y-%m-%d"), "%Y-%m-%d") + timedelta(days=int(delta)))
        #endDate = endDate.strftime("%Y-%m-%d")
        startDate = startDate.strftime("%Y-%m-%d")
        #print("End : ", endDate)
    estLostSale.append(getEstimatedLostSale(row['SKU'],startDate,row['OfflineDate'],row['OfflineDays']))


localtime2 = time.time()
print("total time taken :", localtime2-localtime)

Local current time : 1526534967.5409539
total time taken : 215.55435943603516


In [ ]:
facts['pop2050'] = formattedDf[:500].apply(final_pop,axis=1)

In [47]:
formattedDf[formattedDf.SKU == '10259FA1DDLKCNAFAMZ']

,OfflineDate,OfflineDays,SKU
0,2018-02-15,1,10259FA1DDLKCNAFAMZ
1,2018-03-01,1,10259FA1DDLKCNAFAMZ
2,2018-03-03,1,10259FA1DDLKCNAFAMZ
3,2018-03-06,1,10259FA1DDLKCNAFAMZ
4,2018-03-09,4,10259FA1DDLKCNAFAMZ
5,2018-03-16,1,10259FA1DDLKCNAFAMZ
6,2018-04-05,1,10259FA1DDLKCNAFAMZ
7,2018-04-12,1,10259FA1DDLKCNAFAMZ
8,2018-04-17,1,10259FA1DDLKCNAFAMZ
9,2018-04-19,1,10259FA1DDLKCNAFAMZ


In [95]:
#pd.DataFrame(estLostSale).head()
pd.DataFrame(estLostSale).to_pickle('./lostSaleEstimates',compression='infer', protocol=4)

In [ ]:
temp = pd.concat([formattedDf[:2000].reset_index(),pd.DataFrame(estLostSale).reset_index()], axis=1) 
temp.columns = ['index', 'OfflineDate', 'OfflineDays', 'SKU', 'index', 'EstLostSales']
temp[['EstLostSales','OfflineDate', 'OfflineDays', 'SKU', ]]

In [96]:
pd.DataFrame(estLostSale).shape

(73270, 1)